In [ ]:
import torchaudio
import torch 
import pandas as pd
import numpy as np
import re 
import os 
from torch.nn import functional as F
import random
import math
import pickle
import gc
import torch.nn as nn
from tqdm.notebook import tqdm
import os
import glob
import warnings
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings(action='ignore')

In [ ]:
base_dir = '/home/work/'

In [ ]:
train_csv = pd.read_csv('train.csv')
valid_csv = pd.read_csv('valid.csv')
test_csv = pd.read_csv('all.csv')

In [ ]:
'''
np1 = [base_dir+'mel_dataset1/'+x for x in os.listdir(base_dir+'mel_dataset1')]
np2 = [base_dir+'mel_dataset2/'+x for x in os.listdir(base_dir+'mel_dataset2')]
np3 = [base_dir+'mel_dataset3/'+x for x in os.listdir(base_dir+'mel_dataset3')]
np4 = [base_dir+'mel_dataset4/'+x for x in os.listdir(base_dir+'mel_dataset4')]
np5 = [base_dir+'mel_dataset5/'+x for x in os.listdir(base_dir+'mel_dataset5')]
np6 = [base_dir+'mel_dataset6/'+x for x in os.listdir(base_dir+'mel_dataset6')]
np7 = [base_dir+'mel_dataset7/'+x for x in os.listdir(base_dir+'mel_dataset7')]
np8 = [base_dir+'mel_dataset8/'+x for x in os.listdir(base_dir+'mel_dataset8')]
'''

"\nnp1 = [base_dir+'mel_dataset1/'+x for x in os.listdir(base_dir+'mel_dataset1')]\nnp2 = [base_dir+'mel_dataset2/'+x for x in os.listdir(base_dir+'mel_dataset2')]\nnp3 = [base_dir+'mel_dataset3/'+x for x in os.listdir(base_dir+'mel_dataset3')]\nnp4 = [base_dir+'mel_dataset4/'+x for x in os.listdir(base_dir+'mel_dataset4')]\nnp5 = [base_dir+'mel_dataset5/'+x for x in os.listdir(base_dir+'mel_dataset5')]\nnp6 = [base_dir+'mel_dataset6/'+x for x in os.listdir(base_dir+'mel_dataset6')]\nnp7 = [base_dir+'mel_dataset7/'+x for x in os.listdir(base_dir+'mel_dataset7')]\nnp8 = [base_dir+'mel_dataset8/'+x for x in os.listdir(base_dir+'mel_dataset8')]\n"

In [ ]:
np1 = glob.glob(base_dir+'mel_dataset1/*.npy')
np2 = glob.glob(base_dir+'mel_dataset2/*.npy')
np3 = glob.glob(base_dir+'mel_dataset3/*.npy')
np4 = glob.glob(base_dir+'mel_dataset4/*.npy')
np5 = glob.glob(base_dir+'mel_dataset5/*.npy')
np6 = glob.glob(base_dir+'mel_dataset6/*.npy')
np7 = glob.glob(base_dir+'mel_dataset7/*.npy')
np8 = glob.glob(base_dir+'mel_dataset8/*.npy')

In [ ]:
total = np1+np2+np3+np4+np5+np6+np7+np8

In [ ]:
train_csv['title'] = ''
for idx,vocal_path in enumerate(train_csv['vocal']):
    title = os.path.splitext(vocal_path)[0].split('/')[-1]
    train_csv.loc[idx,'title'] = title

valid_csv['title'] = ''

for idx,vocal_path in enumerate(valid_csv['vocal']):
    title = os.path.splitext(vocal_path)[0].split('/')[-1]
    valid_csv.loc[idx,'title'] = title

test_csv['title'] = ''
for idx,vocal_path in enumerate(test_csv['vocal']):
    title = os.path.splitext(vocal_path)[0].split('/')[-1]
    test_csv.loc[idx,'title'] = title

In [ ]:
total_npy1 = pd.DataFrame(total,columns=['npy_path'])

total_npy1['title'] = ''
for idx,vocal_path in enumerate(total_npy1['npy_path']):
    title = os.path.splitext(vocal_path)[0].split('/')[-1]
    total_npy1.loc[idx,'title'] = title


,npy_path,title
0,/home/work/mel_dataset1/김건모_너에게 (마음으로 하는 말).npy,김건모_너에게 (마음으로 하는 말)
1,/home/work/mel_dataset1/김광석_너에게.npy,김광석_너에게
2,/home/work/mel_dataset1/권인하_사랑을 잃어버린 나.npy,권인하_사랑을 잃어버린 나
3,/home/work/mel_dataset1/XIA (준수)_Fever.npy,XIA (준수)_Fever
4,/home/work/mel_dataset1/권인하_그것만이 내세상.npy,권인하_그것만이 내세상
...,...,...
4496,/home/work/mel_dataset8/KCM_버릇처럼 셋을 센다.npy,KCM_버릇처럼 셋을 센다
4497,/home/work/mel_dataset8/폴킴_Young Selfish.npy,폴킴_Young Selfish
4498,/home/work/mel_dataset8/KCM_멀리있기.npy,KCM_멀리있기
4499,/home/work/mel_dataset8/홍대광_이미 넌 고마운 사람.npy,홍대광_이미 넌 고마운 사람


In [ ]:
d = pd.merge(test_csv, total_npy1, left_on='title', right_on='title', how='left')

In [ ]:
d[d['npy_path'].isnull()==True].sort_values('title')

,vocal,label,title,npy_path
760,"Final_dataset/자우림_스물다섯, 스물하나.wav",4418,"자우림_스물다섯, 스물하나",NaN
4300,Final_dataset/카더가든_밤새 (취향저격 그녀 X 카더가든).wav,4382,카더가든_밤새 (취향저격 그녀 X 카더가든),NaN


In [ ]:
d['npy_path'][d['title'] == '자우림_스물다섯, 스물하나'] = '/home/work/mel_dataset6/자우림_스물다섯_스물하나.npy'
d['npy_path'][d['title'] == '카더가든_밤새 (취향저격 그녀 X 카더가든)'] = '/home/work/mel_dataset7/카더가든_밤새.npy' 

In [ ]:
a = pd.merge(train_csv, d[['title','npy_path']], left_on='title', right_on='title', how='left')

In [ ]:
b = pd.merge(valid_csv, d[['title','npy_path']], left_on='title', right_on='title', how='left')

In [ ]:
a.to_csv('new_train.csv',index=False)
b.to_csv('new_valid.csv',index=False)
d.to_csv('new_all.csv',index=False)